# R1 Q2: Lifetime Risk Comparisons

## Reviewer Question

**Referee #1, Q2**: "Please provide lifetime risk comparisons with established clinical models."

## Why This Matters

Lifetime risk estimates are clinically important for:
- Long-term risk communication with patients
- Comparing model performance across different prediction horizons
- Understanding how risk accumulates over time

## Our Approach

We calculate **remaining lifetime risk** using **30-year dynamic risk** calculated at enrollment:

1. **30-Year Dynamic Risk at Enrollment**: Calculate probability of developing disease over the next 30 years from enrollment age
2. **Remaining Lifetime Risk**: This represents the remaining lifetime risk from enrollment age
3. **Comparison**: Compare with PREVENT (30-year ASCVD risk score)

**Key Insight**: 30-year dynamic risk at enrollment = remaining lifetime risk. This uses **only information present at enrollment** (not updated each year). For predictions that update each year with new information, see `R1_Clinical_Utility_Dynamic_Risk_Updating.ipynb`.


## 1. 30-Year Dynamic Risk at Enrollment

We use **30-year dynamic risk** calculated at enrollment to represent remaining lifetime risk. This uses **only information present at enrollment** (not updated each year). The 30-year horizon represents the remaining lifetime risk from enrollment age.


In [14]:
import torch
import pandas as pd
import numpy as np
from pathlib import Path
import sys
import matplotlib.pyplot as plt

sys.path.append('/Users/sarahurbut/aladynoulli2/pyScripts')

# Load data
base_path = Path('/Users/sarahurbut/Library/CloudStorage/Dropbox-Personal/data_for_running/')
Y = torch.load(base_path / 'Y_tensor.pt', weights_only=False)
E = torch.load(base_path / 'E_matrix.pt', weights_only=False)
essentials = torch.load(base_path / 'model_essentials.pt', weights_only=False)
disease_names = essentials['disease_names']

# Load pce_df for patient demographics
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
pandas2ri.activate()
readRDS = robjects.r['readRDS']
pce_data = readRDS('/Users/sarahurbut/Library/CloudStorage/Dropbox-Personal/pce_df_prevent.rds')
pce_df = pandas2ri.rpy2py(pce_data)

print("="*80)
print("LOADING DATA FOR LIFETIME RISK ANALYSIS")
print("="*80)
print(f"Y shape: {Y.shape}")
print(f"E shape: {E.shape}")
print(f"pce_df shape: {pce_df.shape}")
print(f"Number of diseases: {len(disease_names)}")


LOADING DATA FOR LIFETIME RISK ANALYSIS
Y shape: torch.Size([407878, 348, 52])
E shape: torch.Size([407878, 348])
pce_df shape: (10000, 16)
Number of diseases: 348


## 2. 30-Year Dynamic Risk Results

The 30-year dynamic risk calculated at enrollment (using only enrollment information) represents remaining lifetime risk. Results are stored in `results/time_horizons/pooled_retrospective/30yr_results.csv`.

In [15]:
# Load 30-year dynamic results
results_dir = Path('../../results/time_horizons/pooled_retrospective')
thirty_yr_results = pd.read_csv(results_dir / '30yr_results.csv')

print("="*80)
print("30-YEAR DYNAMIC RISK (REMAINING LIFETIME RISK) AT ENROLLMENT")
print("="*80)
print("\nThis shows 30-year dynamic risk predictions calculated at enrollment.")
print("This uses ONLY information present at enrollment (not updated each year).")
print("This represents 'remaining lifetime risk' - the probability of developing")
print("disease over the next 30 years from enrollment age.")
print("\nResults from time_horizons analysis:")
display(thirty_yr_results.head(15))


30-YEAR DYNAMIC RISK (REMAINING LIFETIME RISK) AT ENROLLMENT

This shows 30-year dynamic risk predictions calculated at enrollment.
This uses ONLY information present at enrollment (not updated each year).
This represents 'remaining lifetime risk' - the probability of developing
disease over the next 30 years from enrollment age.

Results from time_horizons analysis:


,Disease,AUC,CI_lower,CI_upper,N_Events,Event_Rate
0,ASCVD,0.708472,0.700786,0.714630,5779,5.611660
1,Diabetes,0.705281,0.697919,0.712127,5908,5.736925
2,Prostate_Cancer,0.674109,0.659409,0.689501,1019,2.274097
3,Osteoporosis,0.645394,0.632523,0.661106,1450,1.409204
4,Lung_Cancer,0.623635,0.601022,0.643033,402,0.390447
5,All_Cancers,0.623069,0.614573,0.632388,3274,3.179196
6,Parkinsons,0.622248,0.581436,0.662521,142,0.137914
7,Bladder_Cancer,0.616474,0.575188,0.650958,249,0.241921
8,Thyroid_Disorders,0.608763,0.602401,0.616599,4903,4.761026
9,Atrial_Fib,0.601826,0.589173,0.616802,1053,1.025916


In [ ]:
# Check if external scores comparison results exist
comparison_results_dir = Path('../../results/comparisons/pooled_retrospective')
comparison_file = comparison_results_dir / 'external_scores_comparison.csv'

if comparison_file.exists():
    print("="*80)
    print("COMPARISON WITH PREVENT (30-YEAR RISK SCORE)")
    print("="*80)
    comparison_results = pd.read_csv(comparison_file, index_col=0)
    
    # Show ASCVD 30-year comparison
    if 'ASCVD_30yr' in comparison_results.index:
        ascvd_30yr = comparison_results.loc['ASCVD_30yr']
        print("\n30-YEAR ASCVD PREDICTION COMPARISON:")
        print(f"  Aladynoulli AUC: {ascvd_30yr['Aladynoulli_AUC']:.4f} ({ascvd_30yr['Aladynoulli_CI_lower']:.4f}-{ascvd_30yr['Aladynoulli_CI_upper']:.4f})")
        print(f"  PREVENT AUC:     {ascvd_30yr['PREVENT_AUC']:.4f} ({ascvd_30yr['PREVENT_CI_lower']:.4f}-{ascvd_30yr['PREVENT_CI_upper']:.4f})")
        print(f"  Difference:      {ascvd_30yr['Difference']:+.4f}")
        print(f"  N patients:      {ascvd_30yr['N_patients']:.0f}")
        print(f"  N events:         {ascvd_30yr['N_events']:.0f}")
        print("\nKey Finding: Aladynoulli 30-year dynamic risk outperforms PREVENT for ASCVD prediction.")
    else:
        print("\nASCVD 30-year comparison not found in results.")
        print("Available comparisons:", comparison_results.index.tolist())
else:
    print("="*80)
    print("EXTERNAL SCORES COMPARISON")
    print("="*80)
    print("\nTo generate comparison results, run:")
    print("  python compare_with_external_scores.py --approach pooled_retrospective")
    print("\nThis will compare our 30-year dynamic risk with PREVENT for ASCVD.")


COMPARISON WITH PREVENT (30-YEAR RISK SCORE)

30-YEAR ASCVD PREDICTION COMPARISON:
  Aladynoulli AUC: 0.7085 (0.7018-0.7149)
  PREVENT AUC:     0.6501 (0.6440-0.6563)
  Difference:      +0.0584
  N patients:      102982
  N events:         5779

Key Finding: Aladynoulli 30-year dynamic risk outperforms PREVENT for ASCVD prediction.


: 

## 3. Summary and Response

### Key Findings

1. **30-year dynamic risk at enrollment represents remaining lifetime risk**: This uses only information present at enrollment (not updated each year)

2. **Aladynoulli outperforms PREVENT**: For ASCVD, our 30-year dynamic risk shows superior discrimination (AUC 0.7085 vs 0.6501) compared to PREVENT

3. **Clinically meaningful improvement**: The +0.0584 AUC difference represents a substantial improvement in risk prediction

### Response to Reviewer

We provide **remaining lifetime risk** estimates using **30-year dynamic risk** calculated at enrollment:

- **Method**: 30-year dynamic risk = cumulative probability of developing disease over 30 years from enrollment, using only enrollment information
- **Comparison**: Compared with PREVENT (30-year ASCVD risk score)
- **Results**: Aladynoulli shows superior discrimination (AUC 0.7085, 95% CI: 0.7018-0.7149) compared to PREVENT (AUC 0.6501, 95% CI: 0.6440-0.6563)

**Implementation**: 
- Results in `results/time_horizons/pooled_retrospective/30yr_results.csv`
- PREVENT comparison in `results/comparisons/pooled_retrospective/external_scores_comparison.csv`
- Generated by `compare_with_external_scores.py`

**Key Insight**: 30-year dynamic risk at enrollment provides remaining lifetime risk estimates that outperform established clinical risk scores, demonstrating the value of our comprehensive disease prediction model.

---
